# Common Model Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], axis=1, inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.value_counts()

APPLICATION_TYPE  AFFILIATION       CLASSIFICATION  USE_CASE      ORGANIZATION  STATUS  INCOME_AMT     SPECIAL_CONSIDERATIONS  ASK_AMT   IS_SUCCESSFUL
T3                Independent       C1000           Preservation  Trust         1       0              N                       5000      1                3507
                  CompanySponsored  C1000           Preservation  Trust         1       0              N                       5000      0                1379
                                    C1200           Preservation  Association   1       0              N                       5000      0                1122
                  Independent       C1000           Preservation  Trust         1       0              N                       5000      0                1102
                                    C1200           Preservation  Trust         1       0              N                       5000      1                1052
                                                      

# Model 1 - Add more hidden layers

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_value_counts = application_df["APPLICATION_TYPE"].value_counts()
application_value_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`  classification_value_counts[classification_value_counts < 1880].index[:].tolist()
application_types_to_replace = application_value_counts[application_value_counts < 500].index[:].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = application_df['CLASSIFICATION'].value_counts()

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts(dropna=False)[application_df["CLASSIFICATION"].value_counts(dropna=False) > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts < 100].index[:].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS','APPLICATION_TYPE','INCOME_AMT'], dtype='int')
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,...,APPLICATION_TYPE_T8,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evalue Model 1

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train.columns)
hidden_nodes_layer1 = 22
hidden_nodes_layer2 = 11
#hidden_nodes_layer3 = 11
hidden_nodes_layer4 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=num_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 22)                1100      
                                                                 
 dense_1 (Dense)             (None, 11)                253       
                                                                 
 dense_2 (Dense)             (None, 5)                 60        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1419 (5.54 KB)
Trainable params: 1419 (5.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# define filepath to where weights will be saved
filepath = '/content/drive/MyDrive/Deep_Learning/Optimization_model1.h5'

# define modelcheckpoint callback to save model weights every 5 epochs
checkpoint_callback = ModelCheckpoint(filepath, save_weights_only=True, save_freq='epoch', period=5)

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6250 - accuracy: 0.6913
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5708 - accuracy: 0.7285
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5611 - accuracy: 0.7291
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5545 - accuracy: 0.7307
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7303
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7312
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7336
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5483 - accura

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7271 - 486ms/epoch - 2ms/step
Loss: 0.5529491901397705, Accuracy: 0.7271137237548828


In [17]:
# Export our model to HDF5 file
nn.save(filepath)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Model 2 - Drop More Columns

In [26]:
# reimport df
app_df_model_2 = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# drop original columns, plus Afficliation and Special Considerations
app_df_model_2.drop(columns=["EIN", "NAME", "AFFILIATION", "SPECIAL_CONSIDERATIONS"], axis=1, inplace=True)

In [28]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_value_counts = app_df_model_2["APPLICATION_TYPE"].value_counts()

In [31]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`  classification_value_counts[classification_value_counts < 1880].index[:].tolist()
application_types_to_replace = app_value_counts[app_value_counts < 800].index[:].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    app_df_model_2['APPLICATION_TYPE'] = app_df_model_2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
app_df_model_2['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = app_df_model_2['CLASSIFICATION'].value_counts()

In [32]:
# You may find it helpful to look at CLASSIFICATION value counts >1
app_df_model_2['CLASSIFICATION'].value_counts(dropna=False)[app_df_model_2["CLASSIFICATION"].value_counts(dropna=False) > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [34]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts < 1880].index[:].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    app_df_model_2['CLASSIFICATION'] = app_df_model_2['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
app_df_model_2['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
app_df_model_2 = pd.get_dummies(app_df_model_2, columns=['CLASSIFICATION','USE_CASE','ORGANIZATION','APPLICATION_TYPE','INCOME_AMT'], dtype='int')
app_df_model_2.head()

KeyError: "None of [Index(['CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'APPLICATION_TYPE',\n       'INCOME_AMT'],\n      dtype='object')] are in the [columns]"

In [39]:
# Split our preprocessed data into our features and target arrays
y = app_df_model_2['IS_SUCCESSFUL']
X = app_df_model_2.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)

In [40]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train.columns)
hidden_nodes_layer1 = 22
hidden_nodes_layer2 = 11

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=num_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 22)                726       
                                                                 
 dense_6 (Dense)             (None, 11)                253       
                                                                 
 dense_7 (Dense)             (None, 1)                 12        
                                                                 
Total params: 991 (3.87 KB)
Trainable params: 991 (3.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# define filepath to where weights will be saved
filepath = '/content/drive/MyDrive/Deep_Learning/Optimization_Model2.h5'

# define modelcheckpoint callback to save model weights every 5 epochs
checkpoint_callback = ModelCheckpoint(filepath, save_weights_only=True, save_freq='epoch', period=5)

In [43]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [44]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6320 - accuracy: 0.6375
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6169 - accuracy: 0.6508
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6148 - accuracy: 0.6537
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6135 - accuracy: 0.6531
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6127 - accuracy: 0.6549
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6122 - accuracy: 0.6546
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6114 - accuracy: 0.6564
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6112 - accuracy: 0.6546
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6109 - accuracy: 0.6556
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6109 - accura

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6152 - accuracy: 0.6583 - 428ms/epoch - 2ms/step
Loss: 0.615206241607666, Accuracy: 0.6583090424537659


In [46]:
# Export our model to HDF5 file
nn.save(filepath)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Model 3 - Use different activation functions for the hidden layers

In [47]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

In [48]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [49]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train.columns)
hidden_nodes_layer1 = 22
hidden_nodes_layer2 = 11

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=num_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 22)                902       
                                                                 
 dense_9 (Dense)             (None, 11)                253       
                                                                 
 dense_10 (Dense)            (None, 11)                132       
                                                                 
 dense_11 (Dense)            (None, 1)                 12        
                                                                 
Total params: 1299 (5.07 KB)
Trainable params: 1299 (5.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
# define filepath to where weights will be saved
filepath3 = '/content/drive/MyDrive/Deep_Learning/Optimization_mode3.h5'

# define modelcheckpoint callback to save model weights every 5 epochs
checkpoint_callback = ModelCheckpoint(filepath, save_weights_only=True, save_freq='epoch', period=5)

In [51]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [52]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5835 - accuracy: 0.7089
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5595 - accuracy: 0.7305
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5570 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5552 - accuracy: 0.7316
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5541 - accuracy: 0.7328
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5531 - accuracy: 0.7338
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5523 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7332
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7339
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5507 - accura

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5592 - accuracy: 0.7266 - 401ms/epoch - 1ms/step
Loss: 0.559209942817688, Accuracy: 0.7266472578048706


In [54]:
# Export our model to HDF5 file
nn.save(filepath3)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Model 4 - Add name

In [3]:
application_df4 = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df4.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df4.drop(columns=["EIN"], axis=1, inplace=True)

In [5]:
# Determine the number of unique values in each column.
application_df4.value_counts()

NAME                                                            APPLICATION_TYPE  AFFILIATION       CLASSIFICATION  USE_CASE      ORGANIZATION  STATUS  INCOME_AMT   SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL
PARENT BOOSTER USA INC                                          T3                CompanySponsored  C2100           Preservation  Trust         1       0            N                       5000     0                887
TOPS CLUB INC                                                   T3                CompanySponsored  C1200           Preservation  Association   1       0            N                       5000     0                672
UNITED STATES BOWLING CONGRESS INC                              T3                CompanySponsored  C1000           Preservation  Trust         1       0            N                       5000     0                583
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                 T3                CompanySponsored  C1200           Preservation  T

In [6]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_value_counts = application_df4["APPLICATION_TYPE"].value_counts()
application_value_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`  classification_value_counts[classification_value_counts < 1880].index[:].tolist()
application_types_to_replace = application_value_counts[application_value_counts < 800].index[:].tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df4['APPLICATION_TYPE'] = application_df4['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df4['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_value_counts = application_df4['CLASSIFICATION'].value_counts()
classification_value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df4['CLASSIFICATION'].value_counts(dropna=False)[application_df4["CLASSIFICATION"].value_counts(dropna=False) > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_value_counts[classification_value_counts < 1880].index[:].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df4['CLASSIFICATION'] = application_df4['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df4['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [12]:
name_value_counts = application_df4["NAME"].value_counts()

In [15]:
# drop lowest value to better visualize data
application_df4['NAME'].value_counts(dropna=False)[application_df4["NAME"].value_counts(dropna=False) > 2]

NAME
PARENT BOOSTER USA INC                                             1260
TOPS CLUB INC                                                       765
UNITED STATES BOWLING CONGRESS INC                                  700
WASHINGTON STATE UNIVERSITY                                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                     408
                                                                   ... 
ENTREPRENEURS ORGANIZATION                                            3
CENTER FOR THE STUDY OF CLASSICAL ARCHITECTURE INC                    3
VETERANS OF FOREIGN WARS OF THE UNITED STATES DEPT OF TENNESSEE       3
AMERICAN MARKETING ASSOCIATION INC                                    3
UNITED STATES BOWLING CONGRESS                                        3
Name: count, Length: 577, dtype: int64

In [16]:
# Choose a cutoff value and create a list of namess to be replaced
# use the variable name `names_to_replace`
names_to_replace = name_value_counts[name_value_counts < 300].index[:].tolist()

# Replace in dataframe
for name in names_to_replace:
    application_df4['NAME'] = application_df4['NAME'].replace(name,"Other")

# Check to make sure replacement was successful
application_df4['NAME'].value_counts()

NAME
Other                                              29662
PARENT BOOSTER USA INC                              1260
TOPS CLUB INC                                        765
UNITED STATES BOWLING CONGRESS INC                   700
WASHINGTON STATE UNIVERSITY                          492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC      408
PTA TEXAS CONGRESS                                   368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC        331
ALPHA PHI SIGMA                                      313
Name: count, dtype: int64

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df4 = pd.get_dummies(application_df4, columns=['NAME', 'AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS','APPLICATION_TYPE','INCOME_AMT'], dtype='int')
application_df4.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_PTA TEXAS CONGRESS,NAME_SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,NAME_TOPS CLUB INC,...,APPLICATION_TYPE_T6,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [20]:
# Split our preprocessed data into our features and target arrays
y = application_df4['IS_SUCCESSFUL']
X = application_df4.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=15)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train.columns)
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=num_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1500      
                                                                 
 dense_1 (Dense)             (None, 15)                465       
                                                                 
 dense_2 (Dense)             (None, 10)                160       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2136 (8.34 KB)
Trainable params: 2136 (8.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# define filepath to where weights will be saved
filepath4 = '/content/drive/MyDrive/Deep_Learning/Optimization_model4.h5'

# define modelcheckpoint callback to save model weights every 5 epochs
checkpoint_callback = ModelCheckpoint(filepath4, save_weights_only=True, save_freq='epoch', period=5)

In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5727 - accuracy: 0.6974
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5299 - accuracy: 0.7376
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5238 - accuracy: 0.7425
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5211 - accuracy: 0.7434
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5196 - accuracy: 0.7452
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5190 - accuracy: 0.7444
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5175 - accuracy: 0.7448
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5174 - accuracy: 0.7459
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5166 - accuracy: 0.7458
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5160 - accura

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5301 - accuracy: 0.7431 - 521ms/epoch - 2ms/step
Loss: 0.5301145315170288, Accuracy: 0.7430903911590576


In [ ]:
# Export our model to HDF5 file
nn.save(filepath4)